###### Load libraries and directories

In [1]:
from IPython.display import display, HTML

In [2]:
# from IPython import get_ipython
from tqdm.notebook import tqdm
import pickle
import os
import pprint
pp = pprint.PrettyPrinter(indent=1)

# Custom modules for debugging
from SliceViewer import ImageSliceViewer3D, ImageSliceViewer3D_1view,ImageSliceViewer3D_2views
from investigate import *

#pd.set_option("display.max_rows", 10)
      
import json
from run_sma_experiment import find_l3_images,output_images
import pprint
from L3_finder import *

# Custom functions
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

Using TensorFlow backend.


In [3]:
get_ipython().run_line_magic('tb', '')

No traceback available to show.


In [4]:
cwd = os.getcwd()
data = '/tf/data'
pickles = '/tf/pickles'
models = '/tf/models'

## Section 1 - Process final images

## Section 2 - Load each study into subject object
<br>
Subject object defined in L3finder.ingest

In [15]:
# Import modules and config file
configfile = os.path.join(cwd,'config/debug_ES/series_filter_ds1.json')
with open(configfile, "r") as f:
        config = json.load(f)

config = config["series_filter"]   
config["new_tim_dicom_dir_structure"] = True
print('Current config dict: ')
pp.pprint(config)

Current config dict: 
{'dicom_dir': '/tf/data',
 'model_path': 'None',
 'new_tim_dicom_dir_structure': True,
 'output_directory': '/tf/output',
 'overwrite': True,
 'save_plots': True,
 'series_to_skip_pickle_file': '/tf/output/broken_sagittal_and_axial_series.pkl',
 'show_plots': False}


In [16]:
# Debug
print("Finding subjects")

subjects = list(
    find_subjects(
        config["dicom_dir"],
        new_tim_dir_structure=config["new_tim_dicom_dir_structure"]
    )
)

print('Subjects found: ', len(subjects))

Finding subjects
Subjects found:  2368


## Section-3 - check if there are subjects with multiple folders (studies)

In [17]:
%%time
# Find series images
print("Finding series")
series = list(flatten(tqdm((s.find_series() for s in subjects),total=len(subjects))))

Finding series



CPU times: user 194 ms, sys: 80.6 ms, total: 275 ms
Wall time: 263 ms


In [18]:
print("Total number of series found: ", len(series))

Total number of series found:  3860


In [19]:
%%time
sagittal_series, axial_series, excluded_series = separate_series(series)

Filtering series
CPU times: user 647 ms, sys: 734 ms, total: 1.38 s
Wall time: 2.38 s


In [24]:
print("Length of valid pats: ", len(subjects))
print("Length of sagittal series", len(sagittal_series))
print("Length of axial series", len(axial_series))
print("Length of excluded series", len(excluded_series))
print("Length of all series in dataset", len(series))

Length of valid pats:  2368
Length of sagittal series 1446
Length of axial series 2413
Length of excluded series 0
Length of all series in dataset 3860


In [21]:
constructed_sagittals = construct_series_for_subjects_without_sagittals(
        subjects, sagittal_series, axial_series)

FILTERING OUT 0.5 axials for recons for debugging!


In [22]:
len(constructed_sagittals)

935

In [23]:
1446+935

2381

In [31]:
duplicates = []
uniques = []
for series in axial_series:
    if series.subject.id_ in uniques:
        duplicates.append(series.subject.id_)
    else:
        uniques.append(series.subject.id_)

        
print(len(duplicates))
print(len(uniques))
print(len(subjects))

47
2366
2368


In [47]:
duplicates_sag = []
uniques_sag = []
for series in sagittal_series:
    if series.subject.id_ in uniques_sag:
        duplicates_sag.append(series.subject.id_)
    else:
        uniques_sag.append(series.subject.id_)

        
print(len(duplicates_sag))
print(len(uniques_sag))
print(len(subjects))

9
1437
2368


In [48]:
duplicates_set = set(duplicates+duplicates_sag)
print(len(duplicates_set))

46


In [52]:
df_d1= load_object(os.path.join(pickles,'df_final_dump1_processed.pkl'))
len(df_d1)

2052

In [33]:
dupin_df1 = [dup for dup in duplicates if dup in df_d1['ID'].values]

In [49]:
subject_ids = [s.id_ for s in subjects]
print(len(subject_ids))

2368


In [50]:
subject_ids_set = set(subject_ids)
print(len(subject_ids_set))

2368


In [58]:
df1_miss = [ids for ids in df_d1['ID'].values if ids not in subject_ids_set]
print(len(df1_miss))

0


In [59]:
df2_miss = [ids for ids in df_d2['ID'].values if ids not in subject_ids_set]
print(len(df2_miss))

4


In [51]:
df_d2= load_object(os.path.join(pickles,'df_final_dump_2.pkl'))
len(df_d2)

381

In [53]:
len(df_d1) + len(df_d2)

2433

In [55]:
len(df_d1['ID'].unique())

2052

In [56]:
len(df_d2['ID'].unique())

381

In [38]:
dupin_df2 = [dup for dup in duplicates if dup in df_d2['ID'].values]

In [57]:
display(df_d1[df_d1['Processed']==0])

,ID,Axial,Sagittal,Overlap,MissingScore,PairValidity,AxSlices,SagSlices,QC,Processed


In [44]:
df_1_47 = df_d1[df_d1['ID'].isin(duplicates)]
display(df_1_47.sort_values(by=['ID']))

,ID,Axial,Sagittal,Overlap,MissingScore,PairValidity,AxSlices,SagSlices,QC,Processed
36,Z1017392,Z1017392-SE-14-Axial_Body_Axial_5.000,Z1017392-SE-11-Sagittal_Body_Sagittal_3.000,0.012,1.505,1.517,62,79,hq,1
44,Z1023089,Z1023089-SE-2-Axial_Body_5.0,Z1023089-SE-5-Sagittal_Body_Sagittal_3.000,0.997,1.324,2.321,64,80,hq,1
45,Z1024771,Z1024771-SE-3-Body_5.0,Z1024771-SE-10-Body_Sagittal_3.000,0.989,1.388,2.377,72,76,hq,1
57,Z1035204,Z1035204-SE-5-Axial_Body_5.0,Z1035204-SE-8-Sagittal_Body_Sagittal_3.000,0.994,1.296,2.29,66,81,hq,1
209,Z1154679,Z1154679-SE-2-Axial_Body_5.0,Z1154679-SE-5-Sagittal_Body_Sagittal_3.000,0.997,1.297,2.294,61,67,hq,1
217,Z1165230,Z1165230-SE-2-_,Z1165230-SE-1001-Vitrea_Snapshot_Vol._Vol,0.419,0.937,1.356,55,108,hq,1
284,Z1203420,Z1203420-SE-2-Axial_Body_5.0,Z1203420-SE-5-Sagittal_Body_Sagittal_3.000,0.994,1.224,2.218,66,63,hq,1
324,Z1225663,Z1225663-SE-11-Axial_Body_Axial_5.000,Z1225663-SE-13-Sagittal_Body_Sagittal_3.000,0.009,5.225,5.234,88,101,hq,1
428,Z1295431,Z1295431-SE-15-Axial_Body_Axial_3.000,Z1295431-SE-17-Sagittal_Body_Sagittal_3.000,0.914,1.322,2.236,88,60,hq,1
432,Z1300146,Z1300146-SE-4-Stnd_Pediatric_Axial_Stnd,None,None,1,None,172,None,lq,1


In [45]:
df_2_47 = df_d2[df_d2['ID'].isin(duplicates)]
display(df_2_47.sort_values(by=['ID']))

,ID,Axial,Sagittal,Overlap,MissingScore,PairValidity,AxSlices,SagSlices,AxThick,SagThick
287,Z1017392,Z1017392-SE-4-Axial_Body__5.0,None,False,1.000,1,67,79,5.0,3
112,Z1023089,Z1023089-SE-2-Axial_Body__5.0,Z1023089-SE-5-Sagittal_Body_Sagittal_3.000,0.997,1.000,1.997,64,80,5.0,3
20,Z1024771,Z1024771-SE-3-Body__5.0,Z1024771-SE-10-Body_Sagittal_3.000,0.989,1.000,1.989,72,76,5.0,3
343,Z1035204,Z1035204-SE-5-Axial_Body__5.0,Z1035204-SE-8-Sagittal_Body_Sagittal_3.000,0.994,1.000,1.994,66,81,5.0,3
322,Z1154679,Z1154679-SE-2-Axial_Body__5.0,Z1154679-SE-5-Sagittal_Body_Sagittal_3.000,0.997,1.000,1.997,61,67,5.0,3
109,Z1165230,Z1165230-SE-2-,None,0.419,0.923,1.342,55,108,5.0,0.5
68,Z1203420,Z1203420-SE-2-Axial_Body__5.0,Z1203420-SE-5-Sagittal_Body_Sagittal_3.000,0.994,1.000,1.994,66,63,5.0,3
104,Z1225663,Z1225663-SE-5-Axial_Body__5.0,Z1225663-SE-14-Sagittal_Bone_Sagittal_3.000,0.995,1.000,1.995,86,61,5.0,3
98,Z1295431,Z1295431-SE-12-Axial_Bone_VOLUME_0.5,Z1295431-SE-17-Sagittal_Body_Sagittal_3.000,1,1.000,2,896,60,0.5,3
93,Z1300146,Z1300146-SE-4-Stnd__Pediatric_Axial_Stnd,None,None,1.000,None,172,None,2.0,None
